## Loop to get the Spatial Structure Parameters

In [1]:
using PyCall

using Distributions, Statistics, Distances
using DataFrames
using LightGraphs, SimpleWeightedGraphs
using Random , Printf, Glob

import PyPlot , CSV
import Distances, StatsBase

rootdir = "/home/stephane/Science/cluster/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/plotdens"
ocdir= "$wdir/oc2"

cd(wdir)

In [ ]:
cd(ocdir)
files= glob("*-oc.csv")
cd(wdir)

df = DataFrame(voname= String[], C= Float64[], Cerr= Float64[], s= Float64[], serr= Float64[], m= Float64[],
merr= Float64[])

fitflag= true
noc= 0 ; nfail= 0

for f in files
    if fitflag
        fname= f[1:end-4] ; fvot= f[1:end-7]
        oc= CSV.read("$ocdir/$fname.csv" , delim= ";")
        ocfile= "$ocdir/$fname.csv" ; voname= "$fvot.vot"
        println("## ocfile: $voname")
        println("## fitting the radial profile....")
    
        ntest= 100 ; niter= 30000 ; nburnin= 10000
    
        ## estimating the binning
        nstar= length(oc.X)
        println("## Nstar: $nstar")
        nbin= min(trunc(Int, nstar/10),20)
        println("## NBIN: $nbin")
    
        verbose= false
        θfit, θfiterr, fitfound= spatialParameter(ocfile, ntest, nbin, niter, verbose, nburnin)
    
        println("## Fit:")
        println(θfit)
        println(θfiterr)
        println(" ")
    
        push!(df, [voname,θfit.C , θfiterr.C, θfit.s , θfiterr.s, θfit.m , θfiterr.m])
        
        noc += 1
        if !fitfound
            println("################################ Failing...########### ")
            println("###################################################### ")
            nfail += 1
        end
        
        println("## OCs analyzed: $noc")
        println("## Fit failed: $nfail \n\n")
    end
end

if fitflag 
    CSV.write("test-fitCauchy-dummy.csv", df, delim=";") 
else
    
    println("\n##\n## No fit...\n##")
end

## ocfile: ASCC 10-2.7deg.vot
## fitting the radial profile....
## Nstar: 119
## NBIN: 11
[1.7915301463051434, 3.981178102900319, 3.463624949523278, 2.132773983696599, 0.5971767154350477, 0.6514655077473251, 0.3215566929265639, 0.15924712411601283, 0.14051216833765826, 0.09429106033184965, 0.08531095934786395]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(4.063638229252693, 3.794579203502773, 2.5993334537197406)
GaiaClustering.modelCauchy(0.029704091343566573, 0.06522477955694936, 0.05810845520730417)
 
## OCs analyzed: 1
## Fit failed: 0 


## ocfile: ASCC 100-1.8deg.vot
## fitting the radial profile....
## Nstar: 149
## NBIN: 14
[0.0, 1.3361114218789711, 1.0203032676166692, 0.36439402414881034, 0.48585869886508054, 0.5631544009572524, 0.39242433369871876, 0.26722228437579426, 0.3429590815518214, 0.30685812559899833, 0.1908730602684244, 0.09505931064751581, 0.1603333706254767, 0.040488224905423316]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(0.8326772699832968, 11.

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(2.2370053756314676, 22.43180024670438, 6.184997404656928)
GaiaClustering.modelCauchy(0.04046466414593289, 0.7747424584287697, 0.3649468773183823)
 
## OCs analyzed: 15
## Fit failed: 0 


## ocfile: ASCC 127-3.4deg.vot
## fitting the radial profile....
## Nstar: 171
## NBIN: 17
[4.265361107213402, 2.258132350877683, 1.103975815984645, 0.6810240423281904, 0.3345381260559531, 0.41056951834139677, 0.23160331803873663, 0.13381525042238132, 0.1771084196766808, 0.1452599757874535, 0.0, 0.010908851936607166, 0.08028915025342882, 0.009292725723776457, 0.0, 0.016187328680126747, 0.007603139228544383]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(3.660743879567095, 2.9677614188801, 1.5852566266296644)
GaiaClustering.modelCauchy(0.05738539568795835, 0.05127486381500099, 0.014129352894259489)
 
## OCs analyzed: 16
## Fit failed: 0 


## ocfile: ASCC 16-3.3deg.vot
## fitting the radial profile....
## Nstar: 520
## NBIN: 20
[0.503056607

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(1.6028891328132762, 11.26739333430442, 2.4174517491401666)
GaiaClustering.modelCauchy(0.04291751393111869, 1.0308677607151253, 0.26954070173918754)
 
## OCs analyzed: 28
## Fit failed: 0 


## ocfile: ASCC 33-2.6deg.vot
## fitting the radial profile....
## Nstar: 701
## NBIN: 20
[1.7355080071548448, 2.2778542593907334, 1.3667125556344406, 1.1466749332987365, 0.6267112248059159, 0.6409546617333239, 0.5840651947155726, 0.3687954515204044, 0.4785408107963724, 0.26831867215880817, 0.22726890569884858, 0.11790135918171527, 0.0997917104114036, 0.04017379646191768, 0.022441913885622982, 0.02099404847364731, 0.01643473491623907, 0.009297364324043798, 0.005863202726874487, 0.002781262831978911]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(1.6415731815423258, 20.851385091178162, 4.339897270914783)
GaiaClustering.modelCauchy(0.03565877527756655, 1.2118041863569085, 0.3732711117981901)
 
## OCs analyzed: 29
## Fit failed: 0 


## ocf

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(17.85949646627982, 8.18708930284757, 6.566513986154188)
GaiaClustering.modelCauchy(0.3138313821127137, 0.3059151800562027, 0.3984112219206024)
 
## OCs analyzed: 41
## Fit failed: 0 


## ocfile: ASCC 84-1.0deg.vot
## fitting the radial profile....
## Nstar: 149
## NBIN: 14
[1.3729335687183455, 1.8305780916244605, 1.9907536746416012, 1.9613336695976364, 0.5720556536326439, 0.4056394634849659, 0.23762311766279057, 0.16017558301714033, 0.10095099769987835, 0.03612983075574592, 0.06537778898658789, 0.029846381928659703, 0.027458671374366913, 0.012712347858503196]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(2.4792882195917216, 6.129113206038228, 3.115544802443814)
GaiaClustering.modelCauchy(0.024709713139767275, 0.14106484213554588, 0.09506312850145393)
 
## OCs analyzed: 42
## Fit failed: 0 


## ocfile: ASCC 89-3.8deg.vot
## fitting the radial profile....
## Nstar: 85
## NBIN: 8
[1.4809835057452858, 1.6661064439634465, 1.0

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(78.47014500344643, 0.496002537054098, 1.095679299738306)
GaiaClustering.modelCauchy(6.210538639097382, 0.2313622773448848, 0.12505407180284692)
 
## OCs analyzed: 54
## Fit failed: 0 


## ocfile: Alessi 62-1.4deg.vot
## fitting the radial profile....
## Nstar: 92
## NBIN: 9
[4.0920406836033365, 1.4185741036491568, 0.4910448820324006, 0.2104478065853144, 0.2000553223094965, 0.05952059176150311, 0.012590894411087199, 0.0, 0.009628331020243144]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(4.6197707527351275, 2.350082666243777, 1.8757469145868186)
GaiaClustering.modelCauchy(0.0501840907923639, 0.030438163459514812, 0.017844582154832506)
 
## OCs analyzed: 55
## Fit failed: 0 


## ocfile: Alessi 9-4.5deg.vot
## fitting the radial profile....
## Nstar: 351
## NBIN: 20
[16.38271482727875, 9.89789020814758, 7.3722216722754395, 4.388227185878237, 4.436985265721326, 3.6302606719538164, 2.8354698739520914, 2.3891459123114847, 1.14

## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(5.0890787823682535, 3.6696381594149567, 3.283875331468133)
GaiaClustering.modelCauchy(0.13988828400550157, 0.290439412604662, 0.431609202171041)
 
## OCs analyzed: 68
## Fit failed: 0 


## ocfile: C1331-622-1.0deg.vot
## fitting the radial profile....
## Nstar: 80
## NBIN: 8
[2.0863382469571388, 1.8545228861841234, 1.5299813811019018, 1.2915427243068, 1.0045332300163998, 1.0747803090385264, 0.48146267237472423, 0.2781784329276185]
## OC fitting done.
## Fit:
GaiaClustering.modelCauchy(2.235219325033716, 5.138448148864937, 2.687578928670325)
GaiaClustering.modelCauchy(0.05523222131510601, 0.37830630391196024, 0.33453211114231723)
 
## OCs analyzed: 69
## Fit failed: 0 


## ocfile: CBJC 4-1.0deg.vot
## fitting the radial profile....
## Nstar: 269
## NBIN: 20
[24.54528610318401, 16.363524068789342, 11.603226157868809, 7.650478785408002, 4.462779291488003, 4.259925687329458, 3.947843219393228, 3.27270481375787, 1.8376150023774136, 2.

In [ ]:
csvfile= "test-fitCauchy3.csv"
df= CSV.read(csvfile , delim= ";")


fig= PyPlot.figure(figsize=(10,10))

ax= PyPlot.subplot(321)
PyPlot.grid("on")
ax.set_xscale("log")
ax.set_yscale("log")
PyPlot.xlabel("C")
PyPlot.ylabel("s")
PyPlot.scatter(df.C, df.s , s=4, facecolor="blue" )
PyPlot.errorbar(df.C, df.s, xerr=df.Cerr, yerr=df.serr, linewidth=0.5,ls="none")


ax= PyPlot.subplot(322)
PyPlot.grid("on")
ax.set_xscale("log")
#ax.set_yscale("log")
PyPlot.xlabel("s")
PyPlot.ylabel("m")
PyPlot.scatter(df.s, df.m , s=4, facecolor="red" )
PyPlot.errorbar(df.s, df.m, xerr=df.serr, yerr=df.merr, linewidth=0.5,ls="none")

ax = PyPlot.subplot(323)
PyPlot.grid("on")
nbins = 100
h = PyPlot.hist(df.C,nbins,range = [0,30], color = "k", alpha=0.6 , label = "C")
PyPlot.xlabel("C")

ax = PyPlot.subplot(324)
PyPlot.grid("on")
nbins = 100
h = PyPlot.hist(df.s,nbins,range = [0,30], color = "r", alpha=0.6 , label = "s")
PyPlot.xlabel("s")

ax = PyPlot.subplot(325)
PyPlot.grid("on")
nbins = 100
h = PyPlot.hist(df.m,nbins,range = [0,10], color = "g", alpha=0.6 , label = "s")
PyPlot.xlabel("m")

PyPlot.show()

--------------
## Checking all the fit with data.

Figs saved with data density with Cauchy Fit.

In [ ]:
function _plotFitDensityFig(filelist, θlist, ifig, nrow, ncol, ocdir)
    
    fig= PyPlot.figure(figsize=(15,10))
    
    for i in 1:length(filelist)
        ocname= filelist[i][1:end-4]
        θ= θlist[i]
        ocfile= "$ocdir/$ocname-oc.csv"
        _plotDensity(ocfile, θ,nrow, ncol, i, ocname)
    end
    
    figname= "densityOC-$ifig.png"
    PyPlot.savefig(figname)
    PyPlot.close(fig)
end


function _plotDensity(ocfile, θ, irow, icol, nplot, title)
    oc= CSV.read(ocfile, delim= ";")
    # binned density
    nstar= length(oc.X)
    nbin= min(trunc(Int, nstar/10),20)
    r2d,ρ2d,err2d= density2D(oc.Y, oc.Z, nbin)
    ρ2dfit= model_rad(r2d, θ, fdens1)
    
    ax= PyPlot.subplot(irow, icol, nplot)
    ax.set_xlabel("r [pc]")
    ax.set_ylabel("ρ")
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_xlim(r2d[1]*0.9, r2d[end]*1.1)
    ax.set_ylim(minimum(ρ2d[ρ2d .> 0])*0.3,maximum(ρ2d)*1.5)
    PyPlot.grid("on")
    PyPlot.scatter(r2d, ρ2d , s=4, facecolor="blue" )
    PyPlot.errorbar(r2d, ρ2d, yerr=2 .* err2d, linewidth=0.5)
    PyPlot.plot(r2d, ρ2dfit, "k-", linewidth=1)
    
    ## TEXT
    ax.text(0.02, 0.01, title, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="green", fontsize=15)
    
    val=θ.C ; txt= @sprintf("C= %3.3f",val)
    ax.text(0.02, 0.2, txt, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="red", fontsize=12)
    val=θ.s ; txt= @sprintf("s= %3.3f [pc]",val)
    ax.text(0.02, 0.3, txt, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="red", fontsize=12) 
    val=θ.m ; txt= @sprintf("m= %3.3f",val)
    ax.text(0.02, 0.4, txt, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="red", fontsize=12)
    
    val= nstar ; txt= @sprintf("Nstar= %d",val)
    ax.text(0.02, 0.52, txt, verticalalignment= "bottom" , horizontalalignment="left",
        transform=ax.transAxes, color="blue", fontsize=12)
    
end

In [ ]:
cd(plotdir)

rowperfig= 4
colperfig= 3
plotperfig= rowperfig*colperfig

ifig= 1
plotaccum=0
filevot= [] ; θlist= [] ; θerrlist= []
for item in 1:length(df.voname)
    push!(filevot, df.voname[item])
    θ= GaiaClustering.modelCauchy(df.C[item], df.s[item], df.m[item])
    push!(θlist, θ)
    θerr= GaiaClustering.modelCauchy(df.Cerr[item], df.serr[item], df.merr[item])
    push!(θerrlist, θerr)
    plotaccum += 1
    
    if plotaccum == plotperfig
        _plotFitDensityFig(filevot, θlist, ifig,rowperfig, colperfig , ocdir)
        println("## Fig $ifig done.")
        filevot= [] ; θlist= [] ; θerrlist= []
        plotaccum= 0 ; ifig += 1

    end
end

cd(wdir)